# Potato analysis

We start by loading all the necessary libraries.

In [1]:
import scipy.io
from PIL import Image
from matplotlib import cm
import numpy as np
import cv2
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine,correlation
import pandas as pd
from itertools import combinations
from imutils import paths
from tqdm import tqdm

Now, we define a function that given the path to a mat file returns 9 cropped images and their associated names. We first define a function to sort the contours of a given image. 

In [2]:
def sort_contours(cnts):
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    
    ys = [x[0] for x in boundingBoxes]
    xs = [x[1] for x in boundingBoxes]
    
    sortedIndicesTemp = []
    for i in range(0,9):
        x = np.argmin(xs)
        sortedIndicesTemp.append(x)
        xs[x] = 1000
    
    sortedIndices = []
    tempYs = [ys[sortedIndicesTemp[0]],ys[sortedIndicesTemp[1]],ys[sortedIndicesTemp[2]]]
    for i in range(0,3):
        y = np.argmin(tempYs)
        sortedIndices.append(sortedIndicesTemp[y])
        tempYs[y]=1000
    tempYs = [ys[sortedIndicesTemp[3]],ys[sortedIndicesTemp[4]],ys[sortedIndicesTemp[5]]]
    for i in range(0,3):
        y = np.argmin(tempYs)
        sortedIndices.append(sortedIndicesTemp[3+y])
        tempYs[y]=1000
    tempYs = [ys[sortedIndicesTemp[6]],ys[sortedIndicesTemp[7]],ys[sortedIndicesTemp[8]]]
    for i in range(0,3):
        y = np.argmin(tempYs)
        sortedIndices.append(sortedIndicesTemp[6+y])
        tempYs[y]=1000
    
    cntsFinal = []
    for i in sortedIndices:
        cntsFinal.append(cnts[i])

#     (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
#         key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return cntsFinal,[cv2.boundingRect(c) for c in cntsFinal]

The following function is in charge of loading the image and cropping it. 

In [3]:
def generateNames(imagePath):
    name = imagePath[imagePath.rfind('V')+1:imagePath.rfind('-')]
    firstVariety = int(name[0:3])
    firstPotato = int(name[3])
    lastVariety = int(name[4:7])
    potatos = []
    for i in range(firstVariety,firstVariety+9):
        if(firstVariety<100):
            potatos.append("Variety 0"+str(firstVariety)+". Potato " + str(firstPotato))
        else:
            potatos.append("Variety "+str(firstVariety)+". Potato " + str(firstPotato))
        if(firstPotato==2):
            firstPotato = 1
            firstVariety += 1
        else:
            firstPotato = 2
    
#     sortedPotatos = [potatos[6],potatos[3],potatos[0],potatos[7],potatos[4],potatos[1],
#                      potatos[8],potatos[5],potatos[2]]
    return potatos

In [4]:
generateNames('data/V10821122-nhs.mat')

['Variety 108. Potato 2',
 'Variety 109. Potato 1',
 'Variety 109. Potato 2',
 'Variety 110. Potato 1',
 'Variety 110. Potato 2',
 'Variety 111. Potato 1',
 'Variety 111. Potato 2',
 'Variety 112. Potato 1',
 'Variety 112. Potato 2']

We define a function to clean the image.

In [5]:
def black(x):
    if(x<0.1):
        return 0.0
    else:
        return x

myfunc_vec = np.vectorize(black)

In [6]:
def cropPotatos(imagePath):
    names = generateNames(imagePath)
    mat = scipy.io.loadmat(imagePath)
    myarray = mat['hsImage'][:,:,75]
    my_array2 = myfunc_vec(myarray)
    cnts,_ = cv2.findContours(np.uint8(my_array2*255).copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntsValid = []
    for c in cnts:
        if cv2.contourArea(c) > 100:
            cntsValid.append(c)
    cs,b=sort_contours(cntsValid)
    crops = []
    for c in cs:
        (x, y, w, h) = cv2.boundingRect(c)
        roi = mat['hsImage'][y:y + h, x:x + w]
        crops.append(roi)
    return crops,names

In [7]:
crops,names=cropPotatos('data/V10821122-nhs.mat')

The following definition generates the histograms associated with each potato.

In [7]:
def generateHistograms(image,channels=(32,170),bins=5,annulus=3):
    """
    Generate histograms associated to a given image
    
    Parameters
    ---------------
    image    : array of shape (width,height,channels)
               A cropped image with all the channels.
    channels : pair of integers
               The range of channels that will be taken into account.                
    bins     : int
               The number of bins that will be used for the histogram.
    annulus  : int
               The number of annulus that will be extracted for computing the histograms. 
               
    Returns
    ----------
    histograms : narray
                 Array of size (bins*(channels[1]-channels[0]),annulus) that stores the 
                 histograms for each annulus of each channel.
    
    """
    shape = image.shape[0:2]
    center = (int(shape[1]/2),int(shape[0]/2))
    minChannel = channels[0]
    maxChannel = channels[1]
    
    histograms = []
    
    
    axis0 = (int(shape[1]/2/annulus))
    axis1 = (int(shape[0]/2/annulus))
    for i in range(1,annulus+1):
        histogram = []
        mask = np.zeros(shape, dtype="uint8")
        cv2.ellipse(mask, center, (axis0*i,axis1*i),0,0,360,(255,255,255),-1)
        cv2.ellipse(mask, center, (axis0*(i-1),axis1*(i-1)),0,0,360,(255,255,255),-1)
        
        for j in range(minChannel,maxChannel):
            roi = image[:,:,j]
            maxValue = np.max(roi)
            masked = cv2.bitwise_and(roi, roi, mask=mask)
            hist = np.histogram(masked.flatten(),bins=bins,range=(0.01,maxValue))[0]
            if(hist.sum()==0):
                print(i,j)
                print(hist)
            histogram.append(hist/hist.sum())
        
        histograms.append(np.array(histogram).flatten())
    
    return np.array(histograms)

Finally, we generate a dataframe with the results.

In [9]:
def analyseImage(imagePath,channels=(32,170),bins=5,annulus=3,method='correlation'):
    crops,names=cropPotatos(imagePath)
    results = []
    for crop,name in zip(crops,names):
        combs = list(combinations(range(0,annulus),2))
        result = []
        result.append(name)
        hists = generateHistograms(crop)
        for comb in combs:
            if(method=='pearsonr'):
                result.append(pearsonr(hists[comb[0]],hists[comb[1]])[0])
            if(method=='cosine'):
                result.append(1-cosine(hists[comb[0]],hists[comb[1]]))
            if(method=='correlation'):
                result.append(1-correlation(hists[comb[0]],hists[comb[1]]))
        results.append(result)
    
    df = pd.DataFrame(data=np.array(results),columns=['name']+list(combinations(range(0,annulus),2)))
    return df

In [23]:
#crops,names=cropPotatos('data/V10821122-nhs.mat')
pd.DataFrame(np.array([np.append([name],generateHistograms(crop,annulus=1)[0]) for name,crop in zip(names,crops)]))

,0,1,2,3,4,5,6,7,8,9,...,681,682,683,684,685,686,687,688,689,690
0,Variety 108. Potato 2,0.060240963855421686,0.04483048472961614,0.07228915662650602,0.5553376295881199,0.2673017652003362,0.06103023516237402,0.04339305711086226,0.06550951847704367,0.5198768197088466,...,0.8706968933669186,0.11167086481947942,0.009515813042261404,0.005037783375314861,0.0030786453960257487,0.8970053176602295,0.0859221942345368,0.008956059333893087,0.005317660229499021,0.00279876854184159
1,Variety 109. Potato 1,0.0544811320754717,0.047405660377358494,0.07665094339622641,0.49599056603773584,0.32547169811320753,0.0544811320754717,0.04764150943396227,0.06745283018867924,0.4962264150943396,...,0.05797784586377563,0.6938486919632335,0.20716474192788123,0.03111006363422107,0.009898656610888523,0.06551967947207164,0.7343860476078247,0.15696441197266087,0.032288475135517324,0.010841385811925524
2,Variety 109. Potato 2,0.04558500759750127,0.04153300692216782,0.06567617761269627,0.5205132534188756,0.32669255444875905,0.04794867465811244,0.03798750633125106,0.0665203444200574,0.5363835893972649,...,0.5501181235234559,0.4063449206884914,0.02311846101923726,0.01721228484643942,0.00320620992237597,0.5859915611814346,0.370126582278481,0.021772151898734177,0.017721518987341773,0.004388185654008439
3,Variety 110. Potato 1,0.023752969121140142,0.03001511552580436,0.08572662491902397,0.6326927229540056,0.2278125674800259,0.025253615368012086,0.03000215842866393,0.08461040362616015,0.6477444420461904,...,0.618656877564241,0.371841934787303,0.004966529907147485,0.0012956164975167351,0.0032390412437918376,0.6938687392055267,0.29835924006908465,0.003238341968911917,0.0012953367875647669,0.003238341968911917
4,Variety 110. Potato 2,0.03565692258271764,0.044330228075811114,0.05750080308384195,0.45133311917764213,0.41117892707998716,0.037905557340186316,0.040154192097654993,0.05942820430452939,0.41824606488917443,...,0.019916479280436876,0.24638612271121105,0.6052039832958561,0.12238997751365242,0.006103437198843559,0.015419209765499518,0.19948602634115,0.6045615162222936,0.1680051397365885,0.012528107934468359
5,Variety 111. Potato 1,0.0676149247577819,0.03751803751803752,0.09523809523809523,0.5609152752009895,0.23871366728509585,0.06920700308959835,0.035633367662203916,0.09824922760041195,0.568692070030896,...,0.3524083985179086,0.6352408398517908,0.007822149032523672,0.0037052284890901604,0.0008233841086867024,0.41683820502264307,0.5720461095100865,0.006587072869493619,0.003911074516261836,0.0006175380815150268
6,Variety 111. Potato 2,0.03425052915143352,0.038676159322686164,0.06349817202232057,0.594766211275736,0.26880892822782376,0.035398230088495575,0.03693728357060408,0.05502116198537899,0.5821469796075414,...,0.17602924201616005,0.8026163909195845,0.020200076952674107,0.0009619084263178145,0.00019238168526356292,0.18814928818776452,0.7929973066564063,0.017121969988457098,0.0013466717968449403,0.00038476337052712584
7,Variety 112. Potato 1,0.07179487179487179,0.031135531135531136,0.06098901098901099,0.5492673992673993,0.2868131868131868,0.07237083180652254,0.030047636496885306,0.06760718211799194,0.5771344814950531,...,0.1175932699341624,0.7869422092172641,0.07991953182150695,0.012801755669348939,0.0027432333577176297,0.1110501280643981,0.7768020490303695,0.09330406147091108,0.015367727771679473,0.0034760336626417855
8,Variety 112. Potato 2,0.027594136246047715,0.04656510491520552,0.09284277091118137,0.6096579476861167,0.22334004024144868,0.026731819488358725,0.04311583788444955,0.0617993676343777,0.5659672319632079,...,0.5561943087093992,0.4139120436907157,0.018970968669157803,0.006898534061511929,0.004024144869215292,0.6507617131359587,0.31934463926415635,0.01925840758838747,0.0074734118999712565,0.0031618281115263006


In [10]:
analyseImage('data/V10821122-nhs.mat')

,name,"(0, 1)","(0, 2)","(1, 2)"
0,Variety 108. Potato 2,0.9069979429180997,0.8231309420171073,0.9734151760236607
1,Variety 109. Potato 1,0.8836220413631974,0.7689085537478141,0.9707577470033868
2,Variety 109. Potato 2,0.9264951698337455,0.8148584089484205,0.9600262935167061
3,Variety 110. Potato 1,0.9477032562902344,0.7841774288406529,0.9258706783922217
4,Variety 110. Potato 2,0.8685727562813668,0.7201925726998986,0.9505636354259775
5,Variety 111. Potato 1,0.8573474918123738,0.7907602325522342,0.9873085240521077
6,Variety 111. Potato 2,0.892004187141499,0.7415033297408817,0.9549798766796471
7,Variety 112. Potato 1,0.8762254091571589,0.7060523858222708,0.950308905623025
8,Variety 112. Potato 2,0.9406860542339426,0.856124013348529,0.973847017123348


We can now analyse all the files

In [25]:
files = list(paths.list_files('data',validExts='.mat'))

In [12]:
files.remove('data/V14711492-nhs.mat')

In [13]:
len(files)

22

In [ ]:
dfs = []
for file in tqdm(files):
    dfs.append(analyseImage(file,bins=20,annulus=2))

result = pd.concat(dfs)

In [27]:
dfs = []
for file in tqdm(files):
    crops,names=cropPotatos(file)
    df=pd.DataFrame(np.array([np.append([name],generateHistograms(crop,annulus=1)[0]) for name,crop in zip(names,crops)]))
    dfs.append(df)

result = pd.concat(dfs)

100%|███████████████████████████████████████████████████████████████████████████████████| 23/23 [00:33<00:00,  1.46s/it]


In [28]:
result.to_csv('histograms.csv',index=None)